# import library

In [105]:
import pandas as pd

df_segment_one_segment_two_before_before = pd.read_csv(r'df_segment_one_segment_two_before_before .csv')
df_segment_one_segment_two_before = pd.read_csv(r'df_segment_one_segment_two_before.csv')
df_segment_one_segment_two_after = pd.read_csv(r'df_segment_one_segment_two_after.csv')

### 前提となるデータ
    課金セグメント
    例: 
    segment_one: セグメント1_1, セグメント1_2, セグメント1_3, セグメント1_4, セグメント1_5
    segment_two: セグメント2_1, セグメント2_2, セグメント2_3, セグメント2_4, セグメント2_5

## segment_one × segment_two UU, segment_one × segment_two sales

In [83]:
df_cross_uu_before = pd.crosstab(df_segment_one_segment_two_before['segment_one'], df_segment_one_segment_two_before['segment_two'], values = df_segment_one_segment_two_before['user_id'], aggfunc = 'count')
df_cross_payment_before = pd.crosstab(df_segment_one_segment_two_before['segment_one'], df_segment_one_segment_two_before['segment_two'], values = df_segment_one_segment_two_before['sum_paid'], aggfunc = 'sum')
df_cross_uu_after = pd.crosstab(df_segment_one_segment_two_after['segment_one'], df_segment_one_segment_two_after['segment_two'], values = df_segment_one_segment_two_after['user_id'], aggfunc = 'count')
df_cross_payment_after = pd.crosstab(df_segment_one_segment_two_after['segment_one'], df_segment_one_segment_two_after['segment_two'], values = df_segment_one_segment_two_after['sum_paid'], aggfunc = 'sum')

df_cross_uu_before.to_csv('df_cross_uu_before.csv', encoding = 'shift-jis')
df_cross_payment_before.to_csv('df_cross_payment_before.csv', encoding = 'shift-jis')
df_cross_uu_after.to_csv('df_cross_uu_after.csv', encoding = 'shift-jis')
df_cross_payment_after.to_csv('df_cross_payment_after.csv', encoding = 'shift-jis')

# segment_one_transition(Table)

In [108]:
df_only_segment_one_before = df_segment_one_segment_two_before[['user_id', 'segment_one', 'sum_paid']]
df_only_segment_one_after = df_segment_one_segment_two_after[['user_id', 'segment_one', 'sum_paid']]

df_segment_one = pd.merge(df_only_segment_one_before, df_only_segment_one_after, how = 'inner', on = 'user_id').rename(columns={'segment_one_x': 'segment_one_before', 'segment_one_y': 'segment_one_after', 'sum_paid_x': 'sum_paid_before', 'sum_paid_y': 'span_paid_after'})
df_segment_one = df_segment_one.groupby(['segment_one_before', 'segment_one_after'], as_index = False, dropna = False).agg({'user_id':'count', 'sum_paid_before':'sum', 'span_paid_after':'sum'})
df_segment_one['diff'] = df_segment_one['span_paid_after'] - df_segment_one['sum_paid_before']
df_segment_one = df_segment_one.sort_values(by = 'diff', ascending = False)
df_segment_one

df_segment_one.to_csv('df_segment_one.csv', encoding = 'shift-jis')

# segment_two_transition(Table)

In [107]:
df_only_segment_two_before = df_segment_one_segment_two_before[['user_id', 'segment_two', 'sum_paid']]
df_only_segment_two_after = df_segment_one_segment_two_after[['user_id', 'segment_two', 'sum_paid']]

segment_two = pd.merge(df_only_segment_two_before, df_only_segment_two_after, how = 'inner', on = 'user_id').rename(columns={'segment_two_x': 'gasha_span_before', 'segment_two_y': 'gasha_span_after', 'sum_paid_x': 'sum_paid_before', 'sum_paid_y': 'span_paid_after'})
segment_two = segment_two.groupby(['segment_two_before', 'segment_two_after'], as_index = False, dropna = False).agg({'user_id':'count', 'sum_paid_before':'sum', 'span_paid_after':'sum'})
segment_two['diff'] = segment_two['span_paid_after'] - segment_two['sum_paid_before']
segment_two = segment_two.sort_values(by = 'diff', ascending = False)

segment_two.to_csv('segment_two.csv', encoding = 'shift-jis')

# segment_one_transition(Sankey diagram)

In [113]:
df_before_before = df_segment_one_segment_two_before_before[['user_id', 'segment_one']].rename(columns={'segment_one': 'yyyy年mm月'})
df_before = df_segment_one_segment_two_before[['user_id', 'segment_one']].rename(columns={'segment_one': 'yyyy年mm+1月'})
df_after = df_segment_one_segment_two_after[['user_id', 'segment_one']].rename(columns={'segment_one': 'yyyy年mm+2月'})

df_segment_one_merge = pd.merge(df_before_before, df_before, how = 'inner', on = 'user_id')
df_segment_one_merge = pd.merge(df_segment_one_merge, df_after, how = 'inner', on = 'user_id')

df_segment_one_merge = df_segment_one_merge[['yyyy年mm月', 'yyyy年mm+1月', 'yyyy年mm+2月']]
df_segment_one_merge.to_csv('df_segment_one_merge.csv', index = False, encoding = 'utf-8')

# segment_two_transition(Sankey diagram)

In [114]:
df_before_before_a = df_segment_one_segment_two_before_before[['user_id', 'segment_two']].rename(columns={'gasha_purpose': 'yyyy年mm月'})
df_before_a = df_segment_one_segment_two_before[['user_id', 'segment_two']].rename(columns={'segment_two': 'yyyy年mm+1月'})
df_after_a = df_segment_one_segment_two_before[['user_id', 'segment_two']].rename(columns={'segment_two': 'yyyy年mm+2月'})

df_segment_two_merge = pd.merge(df_before_before_a, df_before_a, how = 'inner', on = 'user_id')
df_segment_two_merge = pd.merge(df_segment_two_merge, df_after_a, how = 'inner', on = 'user_id')

df_segment_two_merge = df_segment_two_merge[['yyyy年mm月', 'yyyy年mm+1月', 'yyyy年mm+2月']]
df_segment_two_merge.to_csv('df_segment_two_merge.csv', index = False,  encoding = 'utf-8')